# Resample Fruad=1 Class
- Train using Keras DNN

In [11]:
import pandas as pd
import numpy as np
import keras

from sklearn.utils import resample, shuffle

from keras.layers import Dense, Dropout, BatchNormalization

from sklearn.metrics import roc_auc_score

from multiprocessing import cpu_count


import xgboost as xgb

!python -c 'import tensorflow as tf; print(tf.version)'

<module 'tensorflow._api.v1.version' from '/home/ec2-user/anaconda3/lib/python3.7/site-packages/tensorflow/_api/v1/version/__init__.py'>


# Read Dataset

In [2]:
def get_string_features(df):
    string_features = []
    for col in df.columns:
        if df[col].dtype == np.dtype('object'):
            string_features.append(col)

    return string_features


def read_data(drop_string_features=True):
    
    df = pd.read_csv('./datasets/kfold/transaction_fold_0_0_0.csv')
    
    if drop_string_features:
        string_features = get_string_features(df)
        df = df.drop(columns=string_features)
    
    return df
    
    
train = read_data(drop_string_features=True)

In [3]:
train.shape

(73816, 380)

# Resample minority class

In [4]:
def upsample_minority(df, random_state=27):
    """
    Upsample minority class (isFraud=1), combine with majority class, and then shuffle them.
    """
    
    minority = df[df['isFraud']==1]
    majority = df[df['isFraud']==0]
    
    minority_upsampled = resample(minority,
                                 replace=True, # sample with replacement
                                 n_samples=len(majority), # the size of Fraud equals to non-Fruad
                                 random_state=random_state)
    
    df = pd.concat([majority, 
                    minority_upsampled])
    
    return shuffle(df, random_state=random_state)


train_resampled = upsample_minority(train)

In [5]:
train_resampled.shape

(142752, 380)

In [6]:
train_resampled.isFraud.value_counts()

1    71376
0    71376
Name: isFraud, dtype: int64

# Preprocess Data

In [7]:
def preprocess(df):
    return df.drop(columns=['isFraud', 'TransactionID', 'TransactionDT']), df['isFraud']


X, y = preprocess(train_resampled)

In [8]:
X = X.fillna(-999)
y = y.astype('int32')

In [12]:
del train
del train_resampled

import gc
gc.collect()

800

# Build Model

In [22]:
def get_model(input_dim, lr=1e-3):
    
#     inputs = keras.layers.Input(shape=[input_dim,])
    
#     x = keras.layers.Dense(256, activation='relu')(inputs)
#     x = keras.layers.Dense(128, activation='relu')(x)
#     x = keras.layers.Dense(64, activation='relu')(x)
#     x = keras.layers.Dense(16, activation='relu')(x)
#     outputs = keras.layers.Dense(1, activation='sigmoid')(x)
    
#     model = keras.models.Model(inputs=inputs,
#                               outputs=outputs)
    

    model = keras.models.Sequential()
    model.add(Dense(256, activation='relu', kernel_initializer = 'uniform', input_dim=input_dim))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(128, activation='relu', kernel_initializer = 'uniform'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu', kernel_initializer = 'uniform'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu', kernel_initializer = 'uniform'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(16, activation='relu', kernel_initializer = 'uniform'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(8, activation='relu', kernel_initializer = 'uniform'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid', kernel_initializer = 'uniform'))
    
    optimizer = keras.optimizers.Adam(lr=lr)

    model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                 metrics=['accuracy'])
    
    
    return model


model = get_model(X.shape[1], lr=1e-3)

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 256)               96768     
_________________________________________________________________
batch_normalization_7 (Batch (None, 256)               1024      
_________________________________________________________________
dropout_7 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 128)               32896     
_________________________________________________________________
batch_normalization_8 (Batch (None, 128)               512       
_________________________________________________________________
dropout_8 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 64)                8256      
__________

# Train

In [23]:
epochs = 50
batch_size = 256

model.fit(X, y,
         epochs=epochs,
         batch_size=batch_size)

Epoch 1/50
142752/142752 [==============================] - 9s 65us/step - loss: 0.5573 - acc: 0.7217
Epoch 2/50
142752/142752 [==============================] - 7s 52us/step - loss: 0.5188 - acc: 0.7455
Epoch 3/50
142752/142752 [==============================] - 8s 53us/step - loss: 0.5109 - acc: 0.7498
Epoch 4/50
142752/142752 [==============================] - 8s 54us/step - loss: 0.5035 - acc: 0.7535
Epoch 5/50
142752/142752 [==============================] - 8s 54us/step - loss: 0.4978 - acc: 0.7586
Epoch 6/50
142752/142752 [==============================] - 8s 54us/step - loss: 0.4920 - acc: 0.7625
Epoch 7/50
142752/142752 [==============================] - 8s 54us/step - loss: 0.4883 - acc: 0.7638
Epoch 8/50
142752/142752 [==============================] - 8s 54us/step - loss: 0.4846 - acc: 0.7664
Epoch 9/50
142752/142752 [==============================] - 8s 53us/step - loss: 0.4782 - acc: 0.7703
Epoch 10/50
142752/142752 [==============================] - 8s 53us/step - loss: 

In [25]:
pred_prob = model.predict_proba(X,
                               batch_size=batch_size)

score = roc_auc_score(y, pred_prob)

print('roc-auc score={}'.format(score))


roc-auc score=0.9119878328578055


In [26]:
epochs = 100
batch_size = 256

model.fit(X, y,
         epochs=epochs,
         batch_size=batch_size)

Epoch 1/100
142752/142752 [==============================] - 7s 52us/step - loss: 0.3875 - acc: 0.8209
Epoch 2/100
142752/142752 [==============================] - 8s 53us/step - loss: 0.3851 - acc: 0.8210
Epoch 3/100
142752/142752 [==============================] - 8s 54us/step - loss: 0.3855 - acc: 0.8216
Epoch 4/100
142752/142752 [==============================] - 8s 53us/step - loss: 0.3822 - acc: 0.8242
Epoch 5/100
142752/142752 [==============================] - 8s 53us/step - loss: 0.3832 - acc: 0.8231
Epoch 6/100
142752/142752 [==============================] - 8s 53us/step - loss: 0.3824 - acc: 0.8234
Epoch 7/100
142752/142752 [==============================] - 8s 53us/step - loss: 0.3797 - acc: 0.8248
Epoch 8/100
142752/142752 [==============================] - 8s 53us/step - loss: 0.3779 - acc: 0.8262
Epoch 9/100
142752/142752 [==============================] - 8s 53us/step - loss: 0.3777 - acc: 0.8264
Epoch 10/100
142752/142752 [==============================] - 8s 53us/ste

142752/142752 [==============================] - 8s 53us/step - loss: 0.3230 - acc: 0.8549
Epoch 80/100
142752/142752 [==============================] - 8s 53us/step - loss: 0.3228 - acc: 0.8560
Epoch 81/100
142752/142752 [==============================] - 8s 53us/step - loss: 0.3228 - acc: 0.8556
Epoch 82/100
142752/142752 [==============================] - 8s 54us/step - loss: 0.3224 - acc: 0.8549
Epoch 83/100
142752/142752 [==============================] - 8s 53us/step - loss: 0.3196 - acc: 0.8568
Epoch 84/100
142752/142752 [==============================] - 8s 53us/step - loss: 0.3199 - acc: 0.8563
Epoch 85/100
142752/142752 [==============================] - 8s 53us/step - loss: 0.3198 - acc: 0.8561
Epoch 86/100
142752/142752 [==============================] - 8s 53us/step - loss: 0.3197 - acc: 0.8569
Epoch 87/100
142752/142752 [==============================] - 8s 53us/step - loss: 0.3174 - acc: 0.8578
Epoch 88/100
142752/142752 [==============================] - 8s 53us/step - 

In [27]:
pred_prob = model.predict_proba(X,
                               batch_size=batch_size)

score = roc_auc_score(y, pred_prob)

print('roc-auc score={}'.format(score))


roc-auc score=0.9688005728358191


In [28]:
epochs = 100
batch_size = 256

model.fit(X, y,
         epochs=epochs,
         batch_size=batch_size)

Epoch 1/100
142752/142752 [==============================] - 8s 55us/step - loss: 0.3087 - acc: 0.8615
Epoch 2/100
142752/142752 [==============================] - 8s 55us/step - loss: 0.3093 - acc: 0.8620
Epoch 3/100
142752/142752 [==============================] - 8s 55us/step - loss: 0.3076 - acc: 0.8617
Epoch 4/100
142752/142752 [==============================] - 8s 54us/step - loss: 0.3088 - acc: 0.8618
Epoch 5/100
142752/142752 [==============================] - 8s 56us/step - loss: 0.3081 - acc: 0.8629
Epoch 6/100
142752/142752 [==============================] - 8s 56us/step - loss: 0.3068 - acc: 0.8624
Epoch 7/100
142752/142752 [==============================] - 8s 57us/step - loss: 0.3065 - acc: 0.8617
Epoch 8/100
142752/142752 [==============================] - 8s 56us/step - loss: 0.3054 - acc: 0.8639
Epoch 9/100
142752/142752 [==============================] - 8s 54us/step - loss: 0.3044 - acc: 0.8634
Epoch 10/100
142752/142752 [==============================] - 8s 53us/ste

142752/142752 [==============================] - 8s 53us/step - loss: 0.2827 - acc: 0.8752
Epoch 80/100
142752/142752 [==============================] - 8s 53us/step - loss: 0.2831 - acc: 0.8753
Epoch 81/100
142752/142752 [==============================] - 8s 53us/step - loss: 0.2822 - acc: 0.8754
Epoch 82/100
142752/142752 [==============================] - 8s 53us/step - loss: 0.2824 - acc: 0.8757
Epoch 83/100
142752/142752 [==============================] - 8s 53us/step - loss: 0.2831 - acc: 0.8754
Epoch 84/100
142752/142752 [==============================] - 8s 54us/step - loss: 0.2794 - acc: 0.8766
Epoch 85/100
142752/142752 [==============================] - 8s 53us/step - loss: 0.2792 - acc: 0.8775
Epoch 86/100
142752/142752 [==============================] - 8s 54us/step - loss: 0.2802 - acc: 0.8764
Epoch 87/100
142752/142752 [==============================] - 8s 53us/step - loss: 0.2789 - acc: 0.8782
Epoch 88/100
142752/142752 [==============================] - 8s 54us/step - 

In [ ]:
pred_prob = model.predict_proba(X,
                               batch_size=batch_size)

score = roc_auc_score(y, pred_prob)

print('roc-auc score={}'.format(score))


roc-auc score=0.9773538543208868


# Train using XGB

In [12]:
seed = 27
model = xgb.XGBClassifier(objective='binary:logistic',
                            n_thread=cpu_count(),
                            seed=seed)

%time model.fit(X, y, verbose=True)

CPU times: user 1min 39s, sys: 1.31 s, total: 1min 41s
Wall time: 1min 41s


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, n_thread=2, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=27, silent=None, subsample=1,
       verbosity=1)

In [14]:
pred_prob = model.predict_proba(X)

score = roc_auc_score(y, pred_prob[:, 1])

print('roc-auc score={}'.format(score))

roc-auc score=0.9118305454399285


In [15]:
X_org, y_org = preprocess(train)

pred_prob = model.predict_proba(X_org)

score = roc_auc_score(y_org, pred_prob[:, 1])

print('roc-auc score={}'.format(score))

roc-auc score=0.9125657594645398
